<a href="https://colab.research.google.com/github/gustvjor2005/nn-tmnist/blob/main/nn_tmnistpynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TMNIST**

Lectura de dataset TMNIST y uso de este en Pytorch
utilizando redes neuronales

1.*Preparando dataset*





In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import TensorDataset
import pandas as pd

In [ ]:
#uso de gpu
torch.manual_seed(2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
#el archivo esta subido a un directorio del drive
dir = '/content/drive/MyDrive/X-Colab/datasets/tmnist/94_character_TMNIST.csv'
df = pd.read_csv(dir)
df_back = pd.read_csv(dir)
df.head(3)
#df.dtypes

,names,labels,1,2,3,4,5,6,7,8,...,775,776,777,778,779,780,781,782,783,784
0,Salsa-Regular,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,MouseMemoirs-Regular,D,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Creepster-Regular,f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#features, eliminamos dos columnas
df_feat=df.drop(['names', 'labels'], axis=1)
#df_feat.head(3)

In [ ]:
#para saber que al numerar los labels
#no haya alteraciones en la data
#ejecutar antes y despues de reemplazar
#los lables
df_back['labels'].value_counts().mean()

2915.8829787234044

In [ ]:
# los lables deber ser enteros 
# reemplazamos los valores de todos los labels
# por numerps de forma secuencial
arr = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z", "A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z", "!", '"', "#", "$", "%", "&", "'", "(", ")", "*", "+", ",", "-", ".", "/", ":", ";", "<", "=", ">", "?", "@", "[", "\\" , "]", "^", "_", "`", "{", "|", "}", "~"]
count=0

for l in arr:
    df_back['labels'].replace(l, count, inplace=True)
    count+=1


In [ ]:
#features como tensor
features = torch.tensor(df_feat.values, dtype=torch.float)
#target como tensor
target = torch.tensor(df_back['labels'].values)
#instanciamos train_dataset
train_dataset = TensorDataset(features, target)

In [ ]:
#instanciamos train_dataloader
train_dataloader = DataLoader(dataset=train_dataset, batch_size=100)

In [ ]:
#mostrar ejemplos de dataloader
i=0
for x, y in train_dataloader:
  i+=1
  if(i<3):
    print(x.shape, y.shape)

torch.Size([100, 784]) torch.Size([100])
torch.Size([100, 784]) torch.Size([100])


2.*Implementando nn's*

In [ ]:
from torch import nn
import torch.optim as optim

In [ ]:
#red de 5 capas
class Net(nn.Module):

  def __init__(self, i_, h1_, h2_,h3_, out_):
    super().__init__()
    self.linear1 = nn.Linear(i_, h1_)
    self.linear2 = nn.Linear(h1_, h2_)
    self.linear3 = nn.Linear(h2_, h3_)
    self.linear4 = nn.Linear(h3_, out_)

  def forward(self, x):
    x = torch.relu(self.linear1(x))
    x = torch.relu(self.linear2(x))
    x = torch.relu(self.linear3(x))
    x = torch.sigmoid(self.linear4(x))
    
    return x

In [ ]:
#funcion de perdida
criterion = nn.CrossEntropyLoss()
#instanciamos modelo
model = Net(784, 300, 500, 300, 94)
#instanciamos optimizador
optimizer = optim.SGD(model.parameters(), lr=.01)

In [ ]:
import time

epochs=300
total_loss=[]
start = time.monotonic()

#modelo general de entrenamiento
for i in range(epochs):
  total=0
  model.to(device)
  for x, y in train_dataloader:
    x, y = x.to(device), y.to(device)
    yhat = model(x)
    loss = criterion(yhat, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    total +=loss.item()
  total_loss.append(total)

end = time.monotonic()

print('span minuts {:>9.2f}'.format((end-start)/60))

In [ ]:
import matplotlib.pyplot as plt

#grafico del costo
plt.plot(total_loss)
plt.show()

In [ ]:
#mostramos la ultima perdida
total_loss[len(total_loss)-1]